In [3]:
# Change to your dataset directory
import os
os.chdir('C:/Users/11421/Desktop/9444/input/breast-histopathology-images')

In [4]:

import torch
# print(torch.cuda.is_available())
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from glob import glob
import fnmatch

In [5]:
# Collect paths to all images
imagePatches = glob('IDC_regular_ps50_idx5/**/*.png', recursive=True)

In [6]:
# Separate image paths based on class
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)

In [7]:
total_num = 277524
count = 0
y = []
progress_increment = total_num * 0.05  # 5% of total
next_progress = progress_increment

for img in imagePatches:
    if img in classZero:
        y.append(0)
    elif img in classOne:
        y.append(1)

    count += 1

    if count >= next_progress:
        print(f'Progress: {count/total_num*100:.2f}%')  # Prints the progress as a percentage
        next_progress += progress_increment


Progress: 5.00%
Progress: 10.00%
Progress: 15.00%
Progress: 20.00%
Progress: 25.00%
Progress: 30.00%
Progress: 35.00%
Progress: 40.00%
Progress: 45.00%
Progress: 50.00%
Progress: 55.00%
Progress: 60.00%
Progress: 65.00%
Progress: 70.00%
Progress: 75.00%
Progress: 80.00%
Progress: 85.00%
Progress: 90.00%
Progress: 95.00%


In [8]:
# Create dataframe
images_df = pd.DataFrame()
images_df["images"] = imagePatches
images_df["labels"] = y

print(images_df)

                                                   images  labels
0       IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001...       0
1       IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001...       0
2       IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001...       0
3       IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001...       0
4       IDC_regular_ps50_idx5\10253\0\10253_idx5_x1001...       0
...                                                   ...     ...
277519  IDC_regular_ps50_idx5\9383\1\9383_idx5_x2051_y...       1
277520  IDC_regular_ps50_idx5\9383\1\9383_idx5_x2051_y...       1
277521  IDC_regular_ps50_idx5\9383\1\9383_idx5_x2101_y...       1
277522  IDC_regular_ps50_idx5\9383\1\9383_idx5_x2101_y...       1
277523  IDC_regular_ps50_idx5\9383\1\9383_idx5_x2101_y...       1

[277524 rows x 2 columns]


In [9]:
# Split data into training and validation sets
train, val = train_test_split(images_df, stratify=images_df.labels, test_size=0.2)

In [10]:
# Dataset class
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224,224))  # Resize images to 224x224
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [11]:
# Model parameters
num_epochs = 10
num_classes = 2
batch_size = 64
learning_rate = 0.002

In [12]:
# Define transforms
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])


In [13]:
# Create datasets and dataloaders
dataset_train = MyDataset(df_data=train, transform=trans_train)
dataset_valid = MyDataset(df_data=val,transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [14]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Define the model
model = models.resnet50(pretrained=False)  # If you want to use the pretrained model, set pretrained=True

# Change the final layer to match the number of classes in the dataset
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Move the model to the GPU if available
model = model.to(device)

C:\Users\11421\.conda\envs\py38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\11421\.conda\envs\py38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [15]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

In [16]:
# Train the model
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/3470], Loss: 0.3730
Epoch [1/10], Step [200/3470], Loss: 0.3083
Epoch [1/10], Step [300/3470], Loss: 0.4439
Epoch [1/10], Step [400/3470], Loss: 0.6410
Epoch [1/10], Step [500/3470], Loss: 0.3960
Epoch [1/10], Step [600/3470], Loss: 0.2396
Epoch [1/10], Step [700/3470], Loss: 0.3523
Epoch [1/10], Step [800/3470], Loss: 0.3942
Epoch [1/10], Step [900/3470], Loss: 0.3250
Epoch [1/10], Step [1000/3470], Loss: 0.3197
Epoch [1/10], Step [1100/3470], Loss: 0.4272
Epoch [1/10], Step [1200/3470], Loss: 0.1821
Epoch [1/10], Step [1300/3470], Loss: 0.3541
Epoch [1/10], Step [1400/3470], Loss: 0.2187
Epoch [1/10], Step [1500/3470], Loss: 0.3795
Epoch [1/10], Step [1600/3470], Loss: 0.3497
Epoch [1/10], Step [1700/3470], Loss: 0.2625
Epoch [1/10], Step [1800/3470], Loss: 0.4990
Epoch [1/10], Step [1900/3470], Loss: 0.3405
Epoch [1/10], Step [2000/3470], Loss: 0.3127
Epoch [1/10], Step [2100/3470], Loss: 0.3303
Epoch [1/10], Step [2200/3470], Loss: 0.5373
Epoch [1/10], Step 

In [17]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
confusion_matrix = torch.zeros(2, 2)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
                 
    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the test images: 90.38104675254482 %


In [18]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet_model.ckpt')